In [1]:
from pathlib import Path
import fasttext
from fasttext import util as ftu

In [2]:
vec_file = Path("/Users/stm/cltk_data/cc.la.300.bin")

In [3]:
vec_file.exists()

True

In [4]:
embeddings = fasttext.load_model(str(vec_file))

In [5]:
embeddings.get_dimension()

300

In [6]:
ftu.reduce_model(embeddings, 100)

In [7]:
embeddings.get_dimension()

100

In [8]:
embeddings.save_model(str(vec_file).replace("300", "100"))

In [6]:
from latin_author_learning.fasttext_wrapper import model_to_vec_str

/Users/stm/Library/Caches/pypoetry/virtualenvs/latin-author-learning-nqRTE_o3-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
vec_str = model_to_vec_str(embeddings)

KeyboardInterrupt: 

In [2]:
from pathlib import Path
import os

In [3]:
cltk_path = Path(os.environ["HOME"]) / "cltk_data" / "lat" / "text"

In [4]:
perseus_path = cltk_path / "lat_text_perseus"

In [5]:
vec_path = cltk_path / "cc.la.100.vec"

In [15]:
with open(vec_path, "w") as f:
    f.write(vec_str)

In [11]:
from latin_author_learning.corpus import Corpus

In [12]:
perseus = Corpus("lat_text_perseus")

In [13]:
perseus.add_data_from_files(
    perseus_path,
    filename_contains="_lat.",
    meta_keys=["fileDesc", "teiHeader"],
    meta_key_prefix="@", 
)

In [14]:
from latin_author_learning.fasttext_wrapper import DatasetWrapper

In [15]:
ds = DatasetWrapper(corpus=perseus, fraction_for_test=0.2, chunksize=100)

In [6]:
trainfile = cltk_path / "perseues_train.txt"
#ds.get_training_data(trainfile)

In [7]:
valid_file = cltk_path / "perseues_valid.txt"
#ds.get_validation_data(valid_file)

In [ ]:
author_model = fasttext.train_supervised(
    input=str(trainfile), dim=100, lr=1.,
    thread=7,
    pretrainedVectors=str(vec_path),
    wordNgrams=2, minn=3, maxn=6,
    epoch=200,
)

Read 3M words
Number of words:  392447
Number of labels: 23
Progress:   3.5% words/sec/thread:  207449 lr:  0.964639 avg.loss:  0.197098 ETA:   0h10m19s

In [24]:
author_model.test(str(valid_file))

(9246, 0.8822193380921479, 0.8822193380921479)

In [8]:
author_model = fasttext.train_supervised(
    input=str(trainfile),
    thread=7,
    pretrainedVectors=str(vec_path),
    dim=100,
    epoch=200,
    autotuneValidationFile=str(valid_file),
    autotuneDuration=600
)

Progress: 100.0% Trials:    3 Best score:  0.866537 ETA:   0h 0m 0s
Training again with best arguments
Read 3M words
Number of words:  392447
Number of labels: 23
Progress: 100.0% words/sec/thread: 2159086 lr:  0.000000 avg.loss:  0.022190 ETA:   0h 0m 0s


In [9]:
author_model.test(str(valid_file))

(9246, 0.8663205710577547, 0.8663205710577547)